In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here


In [5]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.data.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

/Users/yizizhang/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yizizhang/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-u

INFO:tensorflow:loss = 2.3158925, step = 1
INFO:tensorflow:probabilities = [[0.09833252 0.09770586 0.08800362 0.09625459 0.0993846  0.09813415
  0.11635538 0.10539106 0.09011088 0.11032724]
 [0.10684854 0.09516672 0.09634486 0.09239596 0.10863251 0.09675011
  0.09927665 0.09734122 0.09790257 0.10934085]
 [0.09822284 0.09867863 0.10406356 0.09868164 0.09554111 0.10734961
  0.11792623 0.08765302 0.09998817 0.09189519]
 [0.09655155 0.09243765 0.09356144 0.0851806  0.09888312 0.09620095
  0.12201513 0.10558802 0.08860413 0.12097745]
 [0.09555325 0.10597484 0.09784392 0.09942908 0.09944013 0.09403369
  0.1114307  0.09720632 0.08814029 0.11094786]
 [0.09956333 0.0927864  0.08711516 0.095124   0.102436   0.09964221
  0.11197034 0.1057881  0.09698012 0.10859434]
 [0.1061068  0.09035818 0.10172202 0.08824157 0.09648886 0.10772482
  0.11053803 0.09872133 0.09531812 0.10478032]
 [0.09660953 0.08974122 0.0993933  0.09708139 0.10446166 0.1055809
  0.11237667 0.10560198 0.08758645 0.10156692]
 [0.09

INFO:tensorflow:global_step/sec: 5.60975
INFO:tensorflow:probabilities = [[0.09889194 0.09230292 0.09441804 0.10051383 0.09400762 0.0953761
  0.10737915 0.10526004 0.10890249 0.1029479 ]
 [0.1102619  0.08238874 0.09441888 0.10883853 0.09860874 0.09993232
  0.10830003 0.10683946 0.08726402 0.10314738]
 [0.1006165  0.10211218 0.10064089 0.09614646 0.10132857 0.09542964
  0.10616635 0.09142093 0.09834847 0.10779008]
 [0.09971583 0.10171926 0.09795231 0.09458741 0.09544805 0.09504462
  0.11908506 0.09301241 0.09206543 0.11136962]
 [0.10232675 0.09928146 0.10876328 0.0979104  0.09587396 0.10004
  0.10932538 0.10012409 0.08904858 0.09730618]
 [0.10926123 0.08065395 0.10322535 0.11687344 0.09938785 0.10167503
  0.11717638 0.09027162 0.08270162 0.09877362]
 [0.11098182 0.0928821  0.09427975 0.08976025 0.09690677 0.09028437
  0.1072485  0.10622194 0.10342043 0.10801404]
 [0.09919661 0.10039626 0.10408291 0.09377104 0.0982916  0.10327516
  0.09879396 0.09686838 0.09869529 0.1066287 ]
 [0.1041317

INFO:tensorflow:loss = 2.2771113, step = 101 (17.826 sec)
INFO:tensorflow:probabilities = [[0.09743056 0.10151112 0.09887654 0.10086411 0.09694807 0.10546577
  0.10603232 0.09227465 0.10289212 0.09770466]
 [0.11582237 0.09409863 0.08175594 0.1008746  0.10598717 0.10172976
  0.10400376 0.10362042 0.09651563 0.09559181]
 [0.10563028 0.08333401 0.11013471 0.09602516 0.10655592 0.11055658
  0.10069079 0.09783205 0.09951954 0.08972091]
 [0.09909049 0.09959761 0.10344882 0.10254386 0.10190652 0.10134686
  0.10342509 0.09388278 0.09222777 0.10253027]
 [0.10448755 0.1018182  0.09869202 0.0932705  0.09246193 0.09519349
  0.10400836 0.10413417 0.1146917  0.09124214]
 [0.10977075 0.08971419 0.10571821 0.08932696 0.09614097 0.09123909
  0.10636286 0.10255142 0.09742986 0.11174577]
 [0.1049178  0.08619704 0.0977577  0.09809025 0.09654871 0.09319182
  0.10759299 0.10435958 0.10484207 0.10650204]
 [0.09183306 0.09229948 0.10801093 0.09211895 0.10297222 0.10068113
  0.10429467 0.10233819 0.09989938 0.

...

INFO:tensorflow:global_step/sec: 4.80399
INFO:tensorflow:probabilities = [[0.00000005 0.00066585 0.00067563 0.99764156 0.00001287 0.00008286
  0.00000036 0.00036413 0.00041949 0.00013718]
 [0.00003433 0.00000024 0.00065632 0.06273549 0.00000102 0.90016913
  0.00001202 0.0000004  0.03635244 0.0000386 ]
 [0.00090521 0.9678077  0.00783621 0.00259252 0.00162961 0.00218857
  0.00328308 0.00545711 0.00688714 0.00141277]
 [0.00181006 0.00000367 0.00003271 0.0001985  0.00406159 0.00403014
  0.00000406 0.5826799  0.00004692 0.40713245]
 [0.00000051 0.00000041 0.00000539 0.00000417 0.99949    0.00000724
  0.00005669 0.00000649 0.00020795 0.00022106]
 [0.00021558 0.97574437 0.0063815  0.00188971 0.00377699 0.00113722
  0.00336276 0.00501026 0.00209034 0.00039129]
 [0.00000045 0.00002297 0.00044152 0.93535656 0.00000286 0.00185991
  0.00000198 0.00000418 0.06223222 0.00007738]
 [0.00000725 0.0000188  0.00276578 0.00004088 0.046221   0.00002222
  0.95047593 0.00019792 0.00017939 0.00007074]
 [0.000

INFO:tensorflow:loss = 0.2006273, step = 11501 (20.822 sec)
INFO:tensorflow:probabilities = [[0.00019544 0.59939474 0.32728446 0.00181507 0.00050719 0.00248193
  0.06697616 0.00003165 0.00128794 0.00002545]
 [0.00004028 0.01065887 0.00787401 0.8662164  0.00008052 0.09636474
  0.00115322 0.00001301 0.01755392 0.0000451 ]
 [0.9935288  0.00000132 0.00071632 0.00023569 0.00000627 0.00324769
  0.0019699  0.00001113 0.00018218 0.00010071]
 [0.00000374 0.00031089 0.999571   0.00010655 0.         0.00000095
  0.00000004 0.00000017 0.00000664 0.        ]
 [0.00353043 0.00006095 0.98022443 0.00766255 0.00000052 0.00009645
  0.00009346 0.00000059 0.00830911 0.00002153]
 [0.00674685 0.00518211 0.0034599  0.00145629 0.00879842 0.87555504
  0.01449739 0.02045197 0.05247302 0.01137896]
 [0.99983037 0.00000001 0.0000105  0.00000259 0.00000005 0.00002382
  0.00000072 0.00007441 0.00000395 0.00005356]
 [0.00970432 0.00697615 0.00357987 0.00602773 0.00022524 0.5676843
  0.3614062  0.00007642 0.04413131 0

INFO:tensorflow:global_step/sec: 4.04661
INFO:tensorflow:probabilities = [[0.05543182 0.00000306 0.91628695 0.00029144 0.00087293 0.0000016
  0.00893471 0.00013147 0.00330986 0.01473628]
 [0.00000013 0.         0.00000019 0.00006241 0.00000002 0.00000185
  0.         0.9977848  0.00010368 0.0020469 ]
 [0.00000085 0.         0.00000012 0.00000095 0.00000001 0.00000123
  0.         0.9998603  0.00000104 0.00013556]
 [0.9640311  0.00000072 0.00947514 0.00047614 0.00067957 0.00227572
  0.01903371 0.00014391 0.00328876 0.0005954 ]
 [0.00040179 0.0000836  0.0000748  0.00634608 0.02037035 0.23120293
  0.00002107 0.05831625 0.00757977 0.6756034 ]
 [0.15015152 0.0116731  0.00823933 0.00319175 0.00183926 0.4135646
  0.06829518 0.02384617 0.31296912 0.00622993]
 [0.9999145  0.         0.00001061 0.00001714 0.         0.00003867
  0.00000003 0.00001762 0.00000038 0.00000104]
 [0.9998505  0.         0.0000268  0.00000107 0.         0.00012024
  0.00000025 0.00000002 0.0000008  0.00000027]
 [0.00000

INFO:tensorflow:loss = 0.28150833, step = 11601 (24.708 sec)
INFO:tensorflow:probabilities = [[0.00097223 0.00001686 0.00043501 0.00002672 0.799999   0.00041427
  0.0024242  0.0034474  0.00015186 0.1921124 ]
 [0.00000077 0.00000047 0.00001056 0.00000614 0.99037766 0.00001797
  0.00001309 0.00002577 0.00003382 0.00951366]
 [0.00025157 0.00012462 0.00026428 0.00065481 0.2859931  0.00739143
  0.0000521  0.0254429  0.00406057 0.6757646 ]
 [0.00139761 0.96174496 0.00796218 0.004068   0.00142603 0.00084339
  0.00226282 0.00257917 0.01673807 0.00097772]
 [0.05872955 0.01603535 0.10360552 0.12937057 0.01925368 0.39412302
  0.01969949 0.03497027 0.16777171 0.05644081]
 [0.00000788 0.00013913 0.0000197  0.00058962 0.00370237 0.00068868
  0.0000063  0.00115141 0.00189612 0.9917989 ]
 [0.00067827 0.00000013 0.00001718 0.03859793 0.00002698 0.9487271
  0.0001808  0.00000129 0.01104759 0.00072271]
 [0.00383418 0.00152795 0.00145234 0.10348321 0.00045366 0.0239094
  0.00037336 0.00118287 0.8106333  0

KeyboardInterrupt: 